In [1]:
# Load and extract the CPI Monthly data
import json
import pandas as pd

# cpi_data = pd.read_csv("./data/CPI_MONTHLY.csv")
with open("./../data/raw/CPI_MONTHLY.json") as f:
    cpi_raw_data = json.load(f)
    
    # Extract observations
    observations = cpi_raw_data["observations"]

    # Flatten the observations for the DataFrame
    flattened_data = []
    for observation in observations:
        row = {"Date": observation["d"]}
        for key, value in observation.items():
            if key != "d":  # Skip the date field, already added
                row[key] = value.get("v", None)  # Get the value field
        flattened_data.append(row)

    # Create DataFrame
    cpi_data = pd.DataFrame(flattened_data)

    # Extract metadata
    metadata = {
        "groupDetail": cpi_raw_data["groupDetail"],
        "terms": cpi_raw_data["terms"],
        "seriesDetail": cpi_raw_data["seriesDetail"],
    }

# Display the DataFrame and metadata
print("CPI Monthly Data:")
print(cpi_data.head())

print("\nMetadata:")
print(metadata)

CPI Monthly Data:
         Date STATIC_TOTALCPICHANGE CPI_MEDIAN CPI_TRIM STATIC_CPIXFET  \
0  1995-01-01                   0.6        1.7      1.8            2.2   
1  1995-02-01                   1.9        1.8      1.8            1.8   
2  1995-03-01                   2.1        1.8      1.9            2.1   
3  1995-04-01                   2.5        1.9      2.1            2.1   
4  1995-05-01                   2.9        2.1      2.3            2.5   

  CPI_COMMON CPIW V41690973 V41690914 ATOM_V41693242  
0        1.1  1.7      86.6      86.6            NaN  
1        1.2  1.7      87.0      87.0            NaN  
2        1.5  1.9      87.2      87.2            NaN  
3        1.5  2.0      87.5      87.4            NaN  
4        1.9  2.2      87.7      87.7            NaN  

Metadata:
{'groupDetail': {'label': 'Consumer Price Index', 'description': 'Includes CPI-trim, CPI-median, and CPI-common as well as other “core” inflation measures.', 'link': 'https://www.bankofcanada.ca/?

In [2]:
# # Ensuring there are no missing data across the different locations (checking across date field)

# import pandas as pd

# # Specify the file path
# file_path = "./../data/raw/Not Seasonally Adjusted.xlsx"

# # Load all sheets into a dictionary of DataFrames
# sheets = pd.read_excel(file_path, sheet_name=None)

# # Initialize variables
# all_months = set()
# sheet_month_ranges = {}

# # Process each sheet
# for sheet_name, df in sheets.items():
#     # Ensure date column is named consistently and convert to datetime with monthly frequency
#     df.columns = df.columns.str.strip().str.lower()
#     if 'date' in df.columns:
#         # Convert the 'date' column to datetime (formatting to monthly)
#         df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.to_period('M')
#         valid_months = df['date'].dropna().unique()
#         all_months.update(valid_months)
#         sheet_month_ranges[sheet_name] = set(valid_months)
#     else:
#         print(f"Warning: Sheet '{sheet_name}' does not contain a 'date' column.")

# # Find the global min and max months across all sheets
# all_months = sorted(all_months)
# min_month = min(all_months)
# max_month = max(all_months)

# # Generate a complete range of months
# complete_month_range = pd.period_range(start=min_month, end=max_month, freq='M')

# # Identify missing months for each sheet
# missing_months_per_sheet = {
#     sheet_name: sorted(set(complete_month_range) - dates)
#     for sheet_name, dates in sheet_month_ranges.items()
# }

# # Summarize and optionally save results to a file
# output_file = "missing_months_summary.txt"
# with open(output_file, "w") as f:
#     f.write(f"Global Month Range: {min_month} to {max_month}\n\n")
#     for sheet_name, missing_months in missing_months_per_sheet.items():
#         summary = (
#             f"Sheet '{sheet_name}':\n"
#             f" - Total Missing Months: {len(missing_months)}\n"
#             f" - Sample Missing Months: {missing_months[:10]}\n\n"  # Show the first 10 missing months
#         )
#         f.write(summary)
#         print(summary)  # Print a summarized version

# print(f"Detailed results saved to '{output_file}'")


In [3]:
import pandas as pd

# Specify the file path
file_path = "./../data/raw/Not Seasonally Adjusted.xlsx"

# Load all sheets into a dictionary of DataFrames
sheets = pd.read_excel(file_path, sheet_name=None)

# Initialize a list to hold processed DataFrames
dataframes = []

# Process each sheet
for sheet_name, df in sheets.items():
    # Standardize column names
    df.columns = df.columns.str.strip().str.lower()
    # Ensure the date column is in datetime format with monthly frequency
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.to_period('M')
        # Suffix the column names (excluding 'date') with the sheet name
        df = df.rename(
            columns={col: f"{col}_{sheet_name.lower()}" for col in df.columns if col != 'date'}
        )
        dataframes.append(df)
    else:
        print(f"Warning: Sheet '{sheet_name}' does not contain a 'date' column.")

# Merge all DataFrames on the 'date' column
hpi_data = dataframes[0]  # Start with the first DataFrame
for df in dataframes[1:]:
    hpi_data = pd.merge(hpi_data, df, on='date', how='outer')

# Reset index for a clean DataFrame
hpi_data = hpi_data.reset_index(drop=True)

# Display the merged DataFrame
print("Merged DataFrame:")
print(hpi_data.head())


Merged DataFrame:
      date  composite_hpi_aggregate  single_family_hpi_aggregate  \
0  2005-01                    100.0                        100.0   
1  2005-02                    101.0                        101.0   
2  2005-03                    102.1                        102.1   
3  2005-04                    103.1                        103.2   
4  2005-05                    103.8                        103.9   

   one_storey_hpi_aggregate  two_storey_hpi_aggregate  \
0                     100.0                     100.0   
1                     101.2                     100.9   
2                     102.4                     101.9   
3                     103.8                     102.9   
4                     104.7                     103.3   

   townhouse_hpi_aggregate  apartment_hpi_aggregate  \
0                    100.0                    100.0   
1                    100.9                    101.2   
2                    101.7                    102.2   
3         

In [4]:
# Read the Prime rate history excel file
import pandas as pd

# Specify the file path
file_path = "./../data/raw/Prime Rate History-WOWA.xlsx"

# Read the Excel file
prc_data = pd.read_excel(file_path)

# Convert the 'Date' column to datetime format
prc_data['Date'] = pd.to_datetime(prc_data['Date'], errors='coerce')

# Display the DataFrame
print(prc_data.head())


        Date  Prime Rate  Bank of Canada Overnight Rate
0 1934-12-01        5.50                            2.5
1 1935-12-01        5.21                            2.5
2 1936-12-01        5.00                            2.5
3 1937-12-01        5.00                            2.5
4 1938-12-01        5.00                            2.5


In [5]:
# Join the three data sources by the date field
import pandas as pd

# Function to normalize column names
def normalize_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

# Normalize column names for all DataFrames
prc_data = normalize_column_names(prc_data)
cpi_data = normalize_column_names(cpi_data)
hpi_data = normalize_column_names(hpi_data)

print(hpi_data.head())

# Ensure the 'date' columns are in a consistent format (YYYY-MM)
for df, name in zip([prc_data, cpi_data], ['prc', 'cpi']):
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.to_period('M')  # Convert to YYYY-MM format
    # Append the file name as suffix to the columns except 'date'
    df.rename(columns={col: f"{col}_{name}" for col in df.columns if col != 'date'}, inplace=True)


# Merge the three DataFrames on the 'date' column
merged_df = prc_data.merge(cpi_data, on='date', how='inner').merge(hpi_data, on='date', how='inner')

# Reset index for a clean DataFrame
merged_df = merged_df.reset_index(drop=True)

# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_df.head())

# Optionally save the merged DataFrame to a CSV file
output_path = "./../data/processed/merged_combined_data.csv"
merged_df.to_csv(output_path, index=False)
print(f"Merged data saved to '{output_path}'")


      date  composite_hpi_aggregate  single_family_hpi_aggregate  \
0  2005-01                    100.0                        100.0   
1  2005-02                    101.0                        101.0   
2  2005-03                    102.1                        102.1   
3  2005-04                    103.1                        103.2   
4  2005-05                    103.8                        103.9   

   one_storey_hpi_aggregate  two_storey_hpi_aggregate  \
0                     100.0                     100.0   
1                     101.2                     100.9   
2                     102.4                     101.9   
3                     103.8                     102.9   
4                     104.7                     103.3   

   townhouse_hpi_aggregate  apartment_hpi_aggregate  \
0                    100.0                    100.0   
1                    100.9                    101.2   
2                    101.7                    102.2   
3                    102.4  

In [6]:
merged_df.shape

(239, 684)

In [7]:
len(merged_df.columns)

684